Purpose: pipeline so that given a list of CCSs, for each label where the 5p barcode, 3p barcode, and adapter lie 

In [4]:
import CCS_Filter as cf
import numpy as np
import random as rd

In [50]:
#adaptation of CCS_Filter's score alignment, except this time 
#return traceback alignment, not score
#FOR NOW: linear gap penalty d
#parameters: gap penalty d, barcode/reference(vertical), seq y(horizontal),
    #score matrix, reference character
#reference character: when returning alignment, replace all characters in seq y
    # with the reference character. Then only return the line of the alignment for 
    # the reference
    #example of return: 55555--5----------------
    #if the reference is the 5p barcode
#initialization: 0s in first row; (i,0) = -i*d
#Traceback favors iy>m>ix becuase more likely to have gaps in ref than 
    #y since reference shorter than y
    

def glocal_alignment(gap_penalty, sequence, reference, score_dict, reference_char):
    gap_penalty = np.abs(gap_penalty)
    #matrix
    num_rows = len(reference) +1
    num_cols = len(sequence) +1
    F = np.zeros((num_rows, num_cols))
    T = np.zeros((num_rows,num_cols)) #traceback
    #labels for tracing, prioritizing iy>m>ix
    IY, M, IX = 3, 2, 1
    T_list = [IY, M, IX]
    #initialization\",
    #row 0 already initialized to zeros
    #keep T[0][j] as zeros because can stop anywhere there
    for i in np.arange(1,num_rows):
        F[i][0] = -1*i*gap_penalty
        T[i][0] = IX
        

    for i in np.arange(1,num_rows): # i represents the row
        for j in np.arange(1,num_cols): #j represents the col
#            print('i is', i)
#            print('j is', j)
#            print("looking at ref(x) char", reference[i-1])
#            print("looking at seq(y) char", sequence[j-1])
            m = F[i-1][j-1] + score_dict[(reference[i-1],sequence[j-1])]
            ix = F[i-1][j] - gap_penalty
            iy = F[i][j-1] - gap_penalty
#            print('m is', m)
#            print('ix is',ix)
#            print('iy is', iy)
            F[i][j], T[i][j] = max(list(zip([iy,m,ix],T_list)))
#            print("T entry is: ",T[i][j])
    #now matrix is filled"
    trace_start_col = np.argmax(F[num_rows-1])
    print("T is: ")
    print(T)
    
    #start traceback
    x_align = [] #reference
    y_align = [] #seq
    seq_length = len(sequence)
    y = sequence
    x = [reference_char]*len(reference)
    i_fin, j_fin = num_rows-1, trace_start_col
    i = i_fin
    j = j_fin
    traceback = T[i][j]
    current = ''
    if j_fin < seq_length:
        for n in np.arange(seq_length - j_fin):
            x_align.append('-')
            y_align.append(y[seq_length-n-1])
    if traceback == M:
        i -= 1
        j -= 1
        y_align.append(y[j])
        x_align.append(x[i])
    elif current == IY:
        j -= 1
        x_align.append('-')
        y_align.append(y[j])
    else: #current == IX
        i -= 1
        x_align.append(x[i])
        y_align.append('-')
    
    
    current = T[i][j]
    
    while i>0 and j>0:
        if current == M:
            i -= 1
            j -= 1
            y_align.append(y[j])
            x_align.append(x[i])
        elif current == IY:
            j -= 1
            x_align.append('-')
            y_align.append(y[j])
        else: #current == IX
            i -= 1
            x_align.append(x[i])
            y_align.append('-')
        current = T[i][j]
    
    if j >0:
        while j >0:
            j -= 1
            x_align.append('-')
            y_align.append(y[j])
            
        
    x_align.reverse()
    y_align.reverse()
    
#    x_align_str = "".join(x_align)
 #   y_align_str = "".join(y_align)
    
    
#    print('y(seq): ', y_align)
#    print('x(ref): ', x_align)
    return x_align

In [52]:
#test strings
reference = "ACGT"
seq_0 = "ACGATAGA"
seq_1 = "AAGACAGT"
seq_2 = "AGGACGCTAA"
#gap_penalty, sequence, reference, score_dict, reference_char
glocal_alignment(gap_penalty=-1, sequence = seq_2, reference = reference, 
                score_dict = cf.score_dict, reference_char = 'R')


T is: 
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 2. 3. 3. 2. 3. 3. 1. 1. 2. 2.]
 [1. 1. 3. 3. 1. 2. 3. 3. 3. 1. 1.]
 [1. 1. 2. 3. 3. 1. 2. 3. 3. 3. 3.]
 [1. 1. 1. 3. 3. 1. 1. 3. 2. 3. 3.]]
y(seq):  ['A', 'G', 'G', 'A', 'C', 'G', 'C', 'T', 'A', 'A']
x(ref):  ['-', '-', '-', 'R', 'R', 'R', '-', 'R', '-', '-']
